# 1 Pakete importieren

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from stargazer.stargazer import Stargazer
import statsmodels.formula.api as smf 
import statsmodels.api as sm
%matplotlib inline
import warnings
from scipy.stats import linregress
from scipy import stats


# 2 CSV-Dateien einlesen

In [2]:
csv_1 = "data/kunden.csv"
csv_2 = "data/besucher.csv"

In [3]:
df_1 = pd.read_csv(csv_1, sep = ";")
df_2 = pd.read_csv(csv_2, sep = ";")

In [4]:
df_1

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1.0,43.0,K0310
1,54,77644.0,103145.70,1.0,40.0,K1042
2,55,44341.0,80565.16,0.0,37.0,K0382
3,49,67271.0,83949.89,0.0,42.0,K0498
4,46,49832.0,93781.58,0.0,41.0,K0552
...,...,...,...,...,...,...
1099,70,69581.0,103751.58,1.0,51.0,K0467
1100,38,48301.0,64230.98,1.0,33.0,K0122
1101,67,63547.0,103123.57,1.0,41.0,K1045
1102,37,32686.0,49215.87,1.0,36.0,K1096


In [5]:
df_2

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
0,30,"39919,0953348874",0,"29,0",K1814
1,41,"53430,67308587748",0,"27,0",K1544
2,64,"73163,2169267002",0,"32,0",K1945
3,41,"36761,482581315344",0,"44,0",K1825
4,35,"42684,33163178136",1,"33,0",K1144
...,...,...,...,...,...
886,32,"34391,82496545645",1,"48,0",K1211
887,41,"68516,49543941053",0,"34,0",K1375
888,65,"89685,33344089528",0,"23,0",K1965
889,68,"95714,30526812703",0,"32,0",K1540


# 3 Überblick über die Datentypen verschaffen

In [6]:
df_1.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht    float64
Zeit          float64
KundeNr        object
dtype: object

In [7]:
df_2.dtypes

Alter          int64
Einkommen     object
Geschlecht     int64
Zeit          object
KundeNr       object
dtype: object

Zu ändern sind später: Einkommen --> float; Geschlecht --> int; Zeit --> float

# 4.1 df_1 Geschlecht

In [8]:
df_1.isnull().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    5
Zeit          0
KundeNr       0
dtype: int64

In [9]:
df_1["Geschlecht"] = df_1["Geschlecht"].fillna(1) # Für 1 (Männlich) entschieden, da die meisten Ausprägungen 1 waren
df_1.isna().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    0
Zeit          0
KundeNr       0
dtype: int64

In [11]:
df_1.describe().T

,count,mean,std,min,25%,50%,75%,max
Alter,1104.0,50.481884,1.222506e+01,30.000000,40.0000,51.000,61.000,1.200000e+02
Einkommen,1104.0,964474.817596,3.009470e+07,-62840.374523,46638.5000,59013.000,70298.000,1.000000e+09
Preis,1104.0,88975.198714,2.446407e+04,21471.650000,70609.4225,88931.455,107093.375,1.654823e+05
Geschlecht,1104.0,0.639493,4.803653e-01,0.000000,0.0000,1.000,1.000,1.000000e+00
Zeit,1104.0,44.375906,1.084804e+01,12.000000,37.0000,44.000,52.000,8.000000e+01


In [12]:
df_2.describe().T

,count,mean,std,min,25%,50%,75%,max
Alter,891.0,49.881033,11.865889,30.0,40.0,49.0,60.0,70.0
Geschlecht,891.0,0.578002,0.494156,0.0,0.0,1.0,1.0,1.0


# df_1 Einkommen

In [ ]:
df_1["Einkommen"].mean()

In [ ]:
Q1 = df_1["Einkommen"].quantile(0.25)
Q3 = df_1["Einkommen"].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
Q1

In [ ]:
Q3

In [ ]:
IQR

In [ ]:
median_einkommen = df_1["Einkommen"].median()
median_einkommen

In [ ]:
df_1["Einkommen"] = df_1["Einkommen"].apply(lambda x: median_einkommen if x <= 0 or x >= 1000000 else x)

In [ ]:
df_1.describe()

# df_1 Alter 

In [ ]:
df_1["Alter"].value_counts()

In [ ]:
modalwert_alter = df_1["Alter"].mode()[0]
modalwert_alter

In [ ]:
anzahl_personen_ueber_70 = len(df_1[df_1["Alter"] > 70])

In [ ]:
anzahl_personen_ueber_70

In [ ]:
df_1["Alter"] = df_1["Alter"].apply(lambda x: modalwert_alter if x >= 100 else x)

In [ ]:
df_1.describe()

# 4.2 df_1 Daten umformen

In [ ]:
df_1["Geschlecht"] = df_1["Geschlecht"].astype("int64")

In [ ]:
df_1.dtypes

In [ ]:
df_1.describe()

# 5  df_2 Daten umformen

In [ ]:
df_2["Einkommen"] = pd.to_numeric(df_2["Einkommen"].str.replace(',', '.'), errors="coerce")

In [ ]:
df_2["Zeit"] = pd.to_numeric(df_2["Zeit"].str.replace(',', '.'), errors="coerce")

In [ ]:
df_2.dtypes

In [ ]:
df_2.describe()

# 6 Zusammenfügen von df_1 und df_2 zu df_3

In [ ]:
df_3 = pd.concat([df_1, df_2],
                axis = 0, # Über die Zeilen zusammenfügen
                ignore_index = True)

In [ ]:
df_3

# 7 Einkommen runden

In [ ]:
df_3["Einkommen"] = df_3["Einkommen"].round(2)
df_3

# 8.1 Textdatei einlesen

In [ ]:
text_datei = "data/geo.txt"
df_text = pd.read_csv(text_datei, header=None, names=["Platzhalter"])
df_text

# 8.2 Überschriften 

In [ ]:
df_text[["KundeNR", "Niederlassung"]] = df_text["Platzhalter"].str.split(expand=True)
df_text

In [ ]:
df_text

In [ ]:
df_text.columns

In [ ]:
df_text = df_text.drop("Platzhalter", axis=1)


df_text.columns

# 9 Im df_3 neue Spalte erstellen

In [ ]:
df_3 = df_3.copy()
df_3["Niederlassung"] = None
df_3

# 10 Spalte umbenennen 

In [ ]:
df_3 = df_3.rename(columns={"KundeNr": "KundeNR"})
df_3.dtypes

# 11 Text-Datei mit CSV mergen

In [ ]:
merged_df = pd.merge(df_3, df_text[["KundeNR", "Niederlassung"]], on="KundeNR", how="left")
merged_df

# 12 Spalte umbennen

In [ ]:
merged_df = merged_df.rename(columns={"Niederlassung_y": "Niederlassung"})
merged_df = merged_df.drop("Niederlassung_x", axis = 1)
merged_df

# 13 Namen der Niederlassungen anzeigen

In [ ]:
merged_df["Niederlassung"].unique()

# 14 Namen der Bundesländer fixen

In [ ]:
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("BERLIN", "Berlin")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Berlin-Mitte", "Berlin")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Berlin-Charlottenburg", "Berlin")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Düsseldorf", "Nordrhein-Westfalen")
merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("NRW", "Nordrhein-Westfalen")

In [ ]:
merged_df["Niederlassung"].unique()

Kunden stammen aus 10 Bundesländern

# 15. 1 Neue Spalte namens "Typ" erstellen (Kunden oder Besucher)

In [ ]:
merged_df["Typ"] = merged_df["Preis"].apply(lambda x: 'Kunde' if pd.notna(x) else 'Besucher')

# 15.2 Spalte umpositionieren

In [ ]:
merged_df.insert(0, "Typ", merged_df.pop("Typ"))
merged_df.describe()

In [ ]:
merged_df["Typ"].value_counts().get("Kunde")

In [ ]:
merged_df["Typ"].value_counts().get("Besucher")

# 16 CSV Speichern

In [ ]:
merged_df.to_csv("data/df_final.csv", index = False)

In [ ]:
df_1.to_csv("data/df_1.csv", index = False)

In [ ]:
df_2.to_csv("data/df_2.csv", index = False)

In [ ]:
df_3.to_csv("data/df_3.csv", index = False)

# 17 Kundennummer als Index

In [ ]:
merged_df.set_index("KundeNR", inplace=True)
merged_df


In [ ]:
for i in merged_df.select_dtypes(include = "number").columns:
    plt.figure(figsize=(15,6))
    sns.boxplot(data=merged_df, x = i)
    plt.show()

# Frage 1: Wie viele Autos wurden verkauft?

In [ ]:
# Zöhle die Anzahl der Preise und gebe die Summe wieder
Sales = merged_df["Preis"].count()
Sales

In [ ]:
Sales_1 = df_1["Preis"].count()
Sales_1

# Frage 2:  Was ist der Höchst-, Mindest- und Durchschnittspreis der verkauften Autos?

In [ ]:
Hoechstpreis = df_1["Preis"].max()
Mindestpreis = df_1["Preis"].min()
Durchschnittspreis = df_1["Preis"].mean().round(2)

In [ ]:
Hoechstpreis

In [ ]:
Mindestpreis

In [ ]:
Durchschnittspreis

In [ ]:
daten = {'': ['Mindestpreis', 'Hoechstpreis', 'Durchschnittspreis'],
        'Preis': [Mindestpreis, Hoechstpreis, Durchschnittspreis]}

df_F2 = pd.DataFrame(daten)


In [ ]:

fig, ax = plt.subplots()
ax.axis('tight')
ax.axis('off')
ax.table(cellText=df_F2.values, colLabels=df_F2.columns, cellLoc = 'center', loc='center')

plt.show()

# Frage 3: Wie hoch war der Gesamtumsatz?


In [ ]:
Gesamtumsatz_1 = df_1["Preis"].sum()
Gesamtumsatz_1

# Frage 4: Wie viele Autos wurden pro Bundesland verkauft?

In [ ]:
# Methode groupby() verwendet, um das DataFrame nach der Spalte "Niederlassung" zu gruppieren
Autos_pro_Bundesland = merged_df.groupby("Niederlassung")["Preis"].count()
Autos_pro_Bundesland

In [ ]:
daten = {"Bundesland": ["Baden-Württemberg", "Bayern", "Berlin", "Brandenburg", "Hessen", "Hamburg", "Niedersachsen", "Nordrhein-Westfalen", "Sachsen", "Thüringen"],
        "Verkaufte_Autos": [205, 188, 126, 37, 96, 43, 96, 175, 89, 49]}
df_torte_daten = pd.DataFrame(daten)
farben_palette = ['lightcoral', 'lightgreen', 'lightskyblue', 'gold', 'orange', 'lightpink', 'lightblue', 'mediumseagreen', 'lightcyan', 'plum']

# Tortendiagramm erstellen
plt.pie(df_torte_daten["Verkaufte_Autos"], labels= None, autopct="%1.1f%%", startangle=90, colors=farben_palette, pctdistance = 0.85)

# Hinzufügen von Beschriftungen und Titel
plt.axis("equal")
plt.title("Verkaufte Autos pro Bundesland", fontsize = 16, fontweight= "bold")
plt.legend(df_torte_daten["Bundesland"], title="Bundesland", bbox_to_anchor=(1, 0.5), loc="center left")
plt.show()

#
df_saeule_daten = pd.DataFrame(daten)

# Säulendiagramm erstellen
plt.figure(figsize=(10, 6))
saeulen = plt.bar(df_saeule_daten["Bundesland"], df_saeule_daten["Verkaufte_Autos"], color='skyblue') 

# Absolute Werte über den Säulen anzeigen
for saeule in saeulen:
    height = saeule.get_height()
    plt.text(saeule.get_x() + saeule.get_width() / 2, height, f"{int(height)}", ha="center", va="top", fontsize=10)
#saeule.get_x() + saeule.get_width() / 2: Dieser Ausdruck gibt die x-Position in der Mitte der aktuellen Säule zurück.
#height: Dies ist die Höhe der aktuellen Säule.
# f"{int(height)}": Formatiert die Höhe der Säule als Ganzzahl und erstellt einen Textstring, der die absolute Zahl repräsentiert

# Hinzufügen von Beschriftungen und Titel
plt.xlabel("Bundesland", fontsize=12)
plt.ylabel("Verkaufte Autos", fontsize=12)
plt.title("Verkaufte Autos pro Bundesland", fontsize=16, fontweight="bold")
plt.xticks(rotation=45, ha="right")
plt.show()

# Frage 5:  Wie hoch war der durchschnittliche Umsatz pro Bundesland?

In [ ]:
# Methode groupby() verwendet, um das DataFrame nach der Spalte "Niederlassung" zu gruppieren
durchschnittsumsatz_pro_bundesland = merged_df.groupby('Niederlassung')['Preis'].mean()
durchschnittsumsatz_pro_bundesland

In [ ]:
# Plot
plt.figure(figsize=(10, 6))

# Balkendiagramm für durchschnittlichen Umsatz pro Auto pro Bundesland
bars = plt.bar(durchschnittsumsatz_pro_bundesland.index, durchschnittsumsatz_pro_bundesland, color='blue', label='Durchschnittsumsatz')

# Zahlen auf den Säulen anzeigen
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Achsentitel und Legende
plt.title('Durchschnittsumsatz pro Auto pro Bundesland')
plt.xlabel('Bundesland')
plt.ylabel('Durchschnittsumsatz pro Auto')

# X-Achsenbeschriftungen senkrecht anzeigen lassen
plt.xticks(rotation='vertical')

plt.show()

# Frage 6: Haben mehr Frauen oder mehr Männer unsere Autos gekauft?

In [ ]:
Kunden_Verteilung = df_1.groupby("Geschlecht").size()

if Kunden_Verteilung.get(1, 0) > Kunden_Verteilung.get(0, 0):
    print("Männer machen die Mehrheit der Kunden aus.")
elif Kunden_Verteilung.get(1, 0) > Kunden_Verteilung.get(0, 0):
     print("Frauen machen die Mehrheit der Kunden aus.")   
else:
    print("50 % Männer und 50 % Frauen.")


In [ ]:
verteilung_nach_geschlecht = df_1["Geschlecht"].value_counts(normalize=True) * 100
anzahl_nach_geschlecht = df_1["Geschlecht"].value_counts()

# Balkendiagramm erstellen
colors = ["blue", "pink"]
plt.bar(["Männlich"], [verteilung_nach_geschlecht[1]], color="blue", label=f"Männlich ({anzahl_nach_geschlecht[1]})\n{verteilung_nach_geschlecht[1]:.2f}%")
plt.bar(["Weiblich"], [verteilung_nach_geschlecht[0]], color="pink", label=f"Weiblich ({anzahl_nach_geschlecht[0]})\n{verteilung_nach_geschlecht[0]:.2f}%")



plt.title("Verteilung der Kunden nach Geschlecht", fontsize = 16, fontweight= "bold")
plt.ylabel("Anteil (%)")
plt.legend()
plt.show()


In [ ]:
Kunden_Verteilung

# Frage 7: Wie hoch ist das Durchschnittsalter unserer Kunden?

In [ ]:
Durchschnittsalter_Kunden_1 = df_1["Alter"].mean().round(1)
Durchschnittsalter_Kunden_1

In [ ]:
# Häufigkeiten zählen und von klein nach groß sortieren
Haeufigkeiten_sortiert = df_1["Alter"].value_counts().sort_index()

# Diagramm erstellen mit Säulen
plt.figure(figsize=(12, 6))
plt.bar(Haeufigkeiten_sortiert.index, Haeufigkeiten_sortiert)

# Diagramm beschriften
plt.title("Alter unserer Kunden")
plt.xlabel("Alter")
plt.ylabel("Häufigkeit")

# Diagramm anzeigen
plt.show()

# Frage 8:  Wie hoch ist das Durchschnittsalter der Besucher in unseren Showrooms?

In [ ]:
Durchschnittsalter_Besucher_1 = df_2["Alter"].mean().round(1)
Durchschnittsalter_Besucher_1 

# Frage 9: Durchschnittsalter männliche Kunden

In [ ]:
Durchschnittsalter_maennlich_Kunden = df_1[df_1['Geschlecht'] == 1]['Alter'].mean().round(2)
Durchschnittsalter_maennlich_Kunden

# Frage 10: Wie hoch ist das Durchschnittseinkommen unserer Kunden?

In [ ]:
Durchschnittseinkommen_Kunden = df_1['Einkommen'].mean().round(2)
Durchschnittseinkommen_Kunden

# Frage 11: Wie hoch ist die Korrelation (Pearson-Korrelation) zwischen den Variablen Alter, Einkommen, Preis und Zeit? (nur für die Kunden)

In [ ]:
df_11 = df_1[["Alter", "Einkommen", "Zeit", "Preis"]]

In [ ]:
df_11.corr()

In [ ]:
sns.heatmap(df_11.corr(), annot = True, cmap = "coolwarm", mask=np.triu(df_11.corr(),+1),linewidth=1.5)
plt.title("Korrelation Heatmap")
plt.show()

# Frage 12:  Wie ist die Variable Zeit verteilt? (Kunden und Besucher zusammen)


In [ ]:
print("Statistische Informationen zur Variable 'Zeit':")
print(merged_df["Zeit"].describe())

In [ ]:
x1 = plt.hist(merged_df["Zeit"], bins=10, edgecolor="black")
plt.title("Verteilung der variablen Zeit", fontsize = 16, fontweight= "bold")
plt.ylabel("Anteil (%)")
plt.xlabel("Zeit (Minuten)")
plt.ylabel("Häufigkeit")
plt.show()


In [ ]:
column_to_plot = "Zeit"

plt.figure(figsize=(15, 6))
sns.kdeplot(data=merged_df, x=column_to_plot)
plt.show()

# Frage 13: Wie viele Kunden haben keinen Kredit bei der Bank genommen?

In [ ]:
# Kunden Jahreseinkommen > Autopreis = kein Bankkredit 
Kunden_ohne_Kredit = df_1[df_1["Einkommen"] > df_1["Preis"]].shape[0]
Kunden_ohne_Kredit

In [ ]:
# Anzeigen der Kunden, die keinen Kredit genommen haben
merged_df.loc[merged_df["Einkommen"] > merged_df["Preis"]]

# Frage 14: Welches sind die sozioökonomischen Merkmale der Kunden, die den Kaufpreis beeinflussen? 


In [ ]:
model1 = smf.ols('Preis ~ Alter + Einkommen + Geschlecht + Zeit', data = merged_df).fit()

In [ ]:
Stargazer([model1])

# Frage 15: Prognostiziere den Kaufpreis für(m, 32, 30000) & (m, 51, 54000) noch nicht richtig

In [ ]:
# Daten vorbereiten
data_for_model = merged_df.dropna(subset=['Preis'])

# Features auswählen und aufbereiten
features = ['Alter', 'Einkommen', 'Geschlecht']
X = data_for_model[features]
y = data_for_model['Preis']

# Konstante zu den Features hinzufügen
X = sm.add_constant(X)

# Lineare Regression erstellen und trainieren
model = sm.OLS(y, X)
result = model.fit()

# Szenarien vorbereiten
szenario_1 = pd.DataFrame([[1, 32, 30000, 1]], columns=['const', 'Alter', 'Einkommen', 'Geschlecht'])
szenario_2 = pd.DataFrame([[1, 51, 54000, 1]], columns=['const', 'Alter', 'Einkommen', 'Geschlecht'])

# Vorhersagen für die Szenarien machen
prediction_1 = result.predict(szenario_1)
prediction_2 = result.predict(szenario_2)



In [ ]:
# Ergebnisse ausgeben
"Ergebnis für m, 32, 30000", prediction_1


In [ ]:
"Ergebnis für m, 51, 54000", prediction_2

# Frage 16: Welche Variable (Ursache) beeinflusst den Preis (Wirkung) des Autos am meisten?

In [ ]:
df_16 = df_1[["Alter", "Einkommen", "Geschlecht", "Preis"]]

In [ ]:
# Standardisierung vornehmen 
df_z = df_16.select_dtypes(include=[np.number]).dropna().apply(stats.zscore)

In [ ]:
# Regression
formel = 'Preis ~ Alter + Einkommen + Geschlecht' 
result = smf.ols(formel, data=df_z).fit()

In [ ]:
# Betas anzeigen lassen
round(result.params, 6)

In [ ]:
result.summary()

Einkommen hat das größte Beta [0;1] und beeinflusst daher den Preis des Autos am meisten

# Frage 17: Schätzen Sie eine Regression, die die Wahrscheinlichkeit des Kaufs eines Autos ermittelt.

In [ ]:
# Daten vorbereiten
merged_df['Kauf'] = (merged_df['Preis'] > merged_df['Preis'].mean()).astype(int)

# Features auswählen und aufbereiten
features = ['Alter', 'Einkommen', 'Geschlecht', 'Zeit']
X = merged_df[features]
y = merged_df['Kauf']

# Konstante zu den Features hinzufügen
X = sm.add_constant(X)

# Logistische Regression erstellen und trainieren
model = sm.Logit(y, X)
result = model.fit()

In [ ]:
# Ergebnisse anzeigen
result.summary()

# Frage 18: Wie hoch ist die Wahrscheinlichkeit, dass Autokauf zustande kommt bei: (m, 32y, 30000€, 30min) & (m, 51y, 54000€, 45min)

In [ ]:
from statsmodels.api import add_constant

# Szenarien vorbereiten
szenario_1 = pd.DataFrame({
    'const': [1],
    'Alter': [32],
    'Einkommen': [30000],
    'Geschlecht': [1],
    'Zeit': [30]
})

szenario_2 = pd.DataFrame({
    'const': [1],
    'Alter': [51],
    'Einkommen': [54000],
    'Geschlecht': [1],
    'Zeit': [45]
})

# Spalten im trainierten Modell
model_columns = result.model.exog_names

# Spalten in den Szenarien anpassen
szenario_1 = szenario_1[model_columns]
szenario_2 = szenario_2[model_columns]

# Vorhersagen für die beiden Szenarien machen
predicted_probabilities_1 = result.predict(szenario_1)
predicted_probabilities_2 = result.predict(szenario_2)

# Wahrscheinlichkeiten in Prozent umrechnen
predicted_probabilities_1 *= 100
predicted_probabilities_2 *= 100

# Ergebnisse ausgeben
print("Wahrscheinlichkeit für Szenario 1:", predicted_probabilities_1)
print("Wahrscheinlichkeit für Szenario 2:", predicted_probabilities_2)

# Frage 19: Auf welche Probleme sind Sie bei der Zusammenführung des Datensatzes gestoßen?

Problem 1: Es fehlen Daten

In [ ]:
#CSV Kunden

#Alter         0
#Einkommen     0
#Preis         0
#Geschlecht    5
#Zeit          0
#KundeNr       0

Problem 2: Behandlung fehlender Daten und Ausreißern

Spalte "Geschlecht" --> 5 fehlende Werte

In [ ]:
# df_1["Geschlecht"] = df_1["Geschlecht"].fillna(1) # Für 1 (Männlich) entschieden, da die meisten Ausprägungen 1 waren

Spalte "Einkommen" --> Positive/Negative Ausreißer und Wert von 0

In [ ]:
#df_1["Einkommen"].mean() --> 964474.8175955408 NICHT GEEIGNET; Mittelwert sehr empfindlich ggü. Ausreißern

In [ ]:
#df_1["Einkommen"].median() --> 59013; Median ist robuster ggü. Ausreißern

In [ ]:
#df_1["Einkommen"] = df_1["Einkommen"].apply(lambda x: median_einkommen if x <= 0 or x >= 1000000 else x)

Spalte "Alter" --> Werte von 100 und 120

In [ ]:
#df_1["Alter"].value_counts()


In [ ]:
#Alter
#55     41
#62     39
#57     34
#66     33
#64     32
#30     32
#34     31
#69     30
#37     30
#31     30
#52     29
#54     29
#58     29
#65     29
#53     29
#61     29
#68     29
#59     28
#35     27
#51     27
#41     26
#40     26
#43     26
#32     26
#46     26
#39     24
#48     24
#38     24
#33     24
#44     23
#50     23
#67     23
#42     22
#70     22
#56     22
#45     22
#49     22
#63     21
#60     20
#47     20
#36     19
#100     1
#120     1

Klare Tendenz, dass bestimmte Altersgruppen besonders häufig auftreten --> deutliche Dominanz einer Altersgruppe erkennbar 

Daher Entschedung für Modus und gegen Median

In [ ]:
#df_1["Alter"].mode()[0]

In [ ]:
#df_1["Alter"] = df_1["Alter"].apply(lambda x: modalwert_alter if x >= 100 else x)

Problem 3: Die Datentypen sind unterschiedlich

In [ ]:
#Datentyp CSV Kunden:

#Alter           int64
#Einkommen     float64
#Preis         float64
#Geschlecht    float64
#Zeit          float64
#KundeNr        object

In [ ]:
#Datentyp CSV Besucher:

#Alter          int64
#Einkommen     object
#Geschlecht     int64
#Zeit          object
#KundeNr       object

Probleme: Datenverlust, Unerwartetes Verhalten

Sicherstellen, dass die Datentypen in den gemeinsamen Spalten zwischen den beiden CSV-Dateien konsistent sind

In [ ]:
#df_1["Geschlecht"] = df_1["Geschlecht"].astype("int64")

In [ ]:
#Datentyp CSV Kunden:

#Alter           int64
#Einkommen     float64
#Preis         float64
#Geschlecht      int64
#Zeit          float64
#KundeNr        object

In [ ]:
#df_2["Einkommen"] = pd.to_numeric(df_2["Einkommen"].str.replace(',', '.'), errors="coerce")

"coerce": anstelle einer Fehlermeldung wird der Wert NaNfür diejenigen Elemente verwendet
die nicht erfolgreich in numerische Werte umgewandelt werden können

In [ ]:
#df_2["Zeit"] = pd.to_numeric(df_2["Zeit"].str.replace(',', '.'), errors="coerce")

In [ ]:
#Datentyp CSV Besucher:

#Alter           int64
#Einkommen     float64
#Geschlecht      int64
#Zeit          float64
#KundeNr        object

Problem 4: Bundesländer nicht einheitlich --> falsche Schreibweise, Stadt, Bezirk

In [ ]:
#merged_df["Niederlassung"].unique()

In [ ]:
#array(['Bayern', 'Berlin', 'Baden-Württemberg', 'Hessen', 'Thüringen',
#       'Sachsen', 'Nordrhein-Westfalen', 'BERLIN', 'Niedersachsen',
#       'Hamburg', 'Brandenburg', 'Berlin-Mitte', 'Düsseldorf', 'NRW',
#       'Berlin-Charlottenburg'], dtype=object)

Zu beheben sind: BERLIN,NRW, Düsseldorf, Berlin-Mitte, Berlin-Charlottenburg

In [ ]:
#merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("BERLIN", "Berlin")
#merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Berlin-Mitte", "Berlin")
#merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Berlin-Charlottenburg", "Berlin")
#merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("Düsseldorf", "Nordrhein-Westfalen")
#merged_df["Niederlassung"] = merged_df["Niederlassung"].replace("NRW", "Nordrhein-Westfalen")

In [ ]:
#merged_df["Niederlassung"].unique()

In [ ]:
#array(['Bayern', 'Berlin', 'Baden-Württemberg', 'Hessen', 'Thüringen',
#       'Sachsen', 'Nordrhein-Westfalen', 'Niedersachsen', 'Hamburg',
#      'Brandenburg'], dtype=object)

# Frage 20: Welche zusätzlichen Informationen sollten gesammelt werden?

Vorstand des Unternehmens ist daran interessiert,
mehr zu erfahren, warum Kunden ihre Autos kaufen, die sozioökonomischen Merkmale 
seiner Kundschaft sowie die Faktoren, die zum Kauf des Autos geführt haben

Vorschlag 1:Kaufverhalten und Praeferenzen 
Welche Faktoren beeinflussen ihre Entscheidung beim Kauf?
Wie wichtig sind bestimmte Produktmerkmale fuer sie?
Welche Kriterien sind bei der Auswahl eines Produkts am entscheidendsten?

Vorschlag 2: Werbung und Marketingkommunikation
Welche Werbekanaele sprechen sie am meisten an?
Wie beeinflusst Werbung ihre Wahrnehmung und Kaufentscheidung?